In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/creditdata/train.csv',parse_dates = ['X1','X2'])

In [ ]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

In [ ]:
categorical_col = [col for col in df.columns if df[col].dtype == 'object']
list_uniq_val = []
for i in categorical_col:
    list_uniq_val.append(df[i].unique())

In [ ]:
# unique_value = {}
unique_value = dict(zip(categorical_col,list_uniq_val))

In [ ]:
unique_value

In [ ]:
import gc
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
def one_hot_encode(df,categorical_col, nan_as_cat = True):
    ori_col = list(df.columns)
    df = pd.get_dummies(df, columns = categorical_col, dummy_na = nan_as_cat)
    categorical_col = [col for col in df.columns if col not in ori_col]
    return df

In [ ]:
# def label_enco(df, categorical_col):
#     for i in categorical_col:
#         df[i], uniques = pd.factorize(df[i])
#     return df[i], categorical_col

In [ ]:
def do_mean(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df


def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [ ]:
group = ['X21','X24','X28','X39','X42']

In [ ]:
def creat_val(df,group):
    
    numerical_col = [col for col in df.columns if (df[col].dtypes =='float64' or df[col].dtypes =='int' )]
    numerical_col.remove('id')
    numerical_col.remove('label')
    for i in numerical_col:
        df = do_mean(df,group,i,i+'_mean')
        df = do_median(df, group, i, i+'_median')
        df = do_std(df, group, i, i+'_std')
        df = do_sum(df, group,i, i+'_sum')
    return df

In [ ]:
df_new = creat_val(df=df,group = group)

In [ ]:
# df_new

In [ ]:
df_new = one_hot_encode(df=df_new, categorical_col= ['X3','X21','X24','X28','X39','X42'])

In [ ]:
import datetime as dt
df_new['time_interval'] = (df_new['X1']-df_new['X2']).dt.days

In [ ]:
df_new.drop(['X1','X2','X22','X33','X34'],axis=1,inplace=True)

In [ ]:
X = df_new.drop(['label','id'], axis=1)
y = df_new['label']

In [ ]:
# pip install pandas --upgrade

In [ ]:
# def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=1000, learning_rate=0.05, output_process=False):
    
#     train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    
#     def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
#         params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
#         params["num_leaves"] = int(round(num_leaves))
#         params['feature_fraction'] = max(min(feature_fraction, 1), 0)
#         params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
#         params['max_depth'] = int(round(max_depth))
#         params['lambda_l1'] = max(lambda_l1, 0)
#         params['lambda_l2'] = max(lambda_l2, 0)
#         params['min_split_gain'] = min_split_gain
#         params['min_child_weight'] = min_child_weight
#         cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
#         return max(cv_result['auc-mean'])
    
#     lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
#                                             'feature_fraction': (0.1, 0.9),
#                                             'bagging_fraction': (0.8, 1),
#                                             'max_depth': (5, 8.99),
#                                             'lambda_l1': (0, 5),
#                                             'lambda_l2': (0, 3),
#                                             'min_split_gain': (0.001, 0.1),
#                                             'min_child_weight': (5, 50)}, random_state=0)
    
#     lgbBO.maximize(init_points=init_round, n_iter=opt_round)
#     return lgbBO.res['max']['max_params']

In [ ]:
# best_param = opt_params = bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=10, 
#                                                   n_folds=3, random_seed=6, n_estimators=100, learning_rate=0.05)

In [ ]:
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.25, random_state =42)
param_grid = {'num_leaves': (24, 45),
            'feature_fraction': (0.1, 0.9),
            'bagging_fraction': (0.8, 1),
            'max_depth': (5, 8.99),
            'lambda_l1': (0, 5),
            'lambda_l2': (0, 3),
            'min_split_gain': (0.001, 0.1),
            'min_child_weight': (5, 50)}
gkf = KFold(n_splits=3, shuffle=True, random_state=42).split(X=X_train, y=y_train)
clf = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=20,
                         learning_rate=0.01, metric='auc')
# grid_search = GridSearchCV(clf, param_grid)
# grid_search.fit(X_train,y_train)
gsearch = GridSearchCV(estimator=clf, param_grid=param_grid, cv=gkf)
lgb_model = gsearch.fit(X=X_train, y=y_train)

y_pred = gsearch.predict(X_test)

print("The best possible combination of hyperparameters: ",gsearch.best_params_)
print("Model accuracy: {:.2f}%".format(accuracy_score(y_pred, y_test)*100))